In [8]:
from extractor import get_text
from scan import get_files
import pandas as pd
import os
import json
from prompt import *
from pathlib import Path
from tqdm.auto import tqdm
import requests


In [9]:
PATH = r"C:\Users\alika\OneDrive\Masaüstü\Profiler\FileProfiler\test\wiki_docs"

In [10]:
directory = Path(PATH)
rows = []
for f in directory.iterdir():
    if f.is_file():
        rows.append({
        "Full Path": str(f.resolve()),
        "File Name": f.stem,
        "Extension": f.suffix.lstrip('.')
        })

df = pd.DataFrame(rows).head(10)

In [11]:
text = get_text(df['Full Path'][0])
question = r'Is there any personal information in the document ?'

In [12]:
headers = {"Authorization": f"Bearer {os.getenv("OPENROUTER_KEY")}",
            "Content-Type": "application/json",}

data = {"model": "qwen/qwen3-8b",
        "messages": [{"role": "user", "content": sp},
                        {"role": "user", "content": up.format(text, question)}],}

r = requests.post(os.getenv("OPENROUTER_URL"),
                headers=headers, data=json.dumps(data))
if r.status_code==200:
        resp = json.loads(r.text.strip())["choices"][0]["message"]["content"]

In [14]:
resp

'{\n"type": "Report",\n"language": "English",\n"answer": "No",\n"reason": "The document discusses the 1939 SANFL season, its ladder, finals series, and grand final. It does not contain any personal data such as names, contact information, or identifying details of individuals."\n}'

'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'

In [14]:
messages = [
    {"role": "system", "content": sp},
    {"role": "user",   "content": up.format(text, question)},
]

    # Turn messages into a single generation prompt
chat_text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,  # adds assistant prefix
)

inputs = tokenizer(chat_text, return_tensors="pt").to(device)

with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        #max_new_tokens=32,
        temperature=0.0,      # for deterministic/classification-like tasks
        top_p=0.9,
        do_sample=False,
    )

# gen_ids = output_ids[0, inputs["input_ids"].shape[1]:]

NameError: name 'tokenizer' is not defined

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Qwen/Qwen3-8B"  # or another Qwen*/Qwen3* instruct model

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
import transformers
print(transformers.__version__)

4.57.1


In [36]:
df_out['content'] = df_out['out_js'].apply(lambda row: row['message']['content'])

In [ ]:
df_out['content'] = df_out['content'].apply(lambda row: json.loads(row))

0    {'type': 'Report', 'language': 'Turkish', 'ans...
1    {'type': 'Report', 'language': 'Turkish', 'ans...
2    {'type': 'Report', 'language': 'Turkish', 'ans...
3    {'type': 'Report', 'language': 'Turkish', 'ans...
4    {'type': 'Report', 'language': 'Turkish', 'ans...
5    {'type': 'Report', 'language': 'Turkish', 'ans...
6    {'type': 'Report', 'language': 'Turkish', 'ans...
7    {'type': 'Report', 'language': 'Turkish', 'ans...
8    {'type': 'Report', 'language': 'Turkish', 'ans...
9    {'type': 'report', 'language': 'tr', 'answer':...
Name: content, dtype: object